In [0]:
%python
##########################################
#  TODO                                  #
#  @author: Mahesh Madhusoodanan Pillai  #
#  @email: mahesh.pillai@databricks.com  #
##########################################

In [0]:
%run "../conf/constants"

In [0]:
%python
sqls = []

databricks = f'''CREATE OR REPLACE TEMPORARY VIEW schema_store_catalog_v as
select "databricks" WAREHOUSE, run_timestamp RUN_TIMESTAMP, iteration_name ITERATION_NAME, concat_ws(".",catalog,db_name,table_name) TABLE_NAME, original_order COL_ORDER, col_name COL_NAME, comment COL_COMMENT, data_type DATA_TYPE from {DATABRICKS_SCHEMA_STORE}'''

netezza = f'''select "netezza" WAREHOUSE, run_timestamp RUN_TIMESTAMP, iteration_name ITERATION_NAME, concat_ws(".",DATABASE,schema,Name) TABLE_NAME, ATTNUM COL_ORDER, ATTNAME COL_NAME, DESCRIPTION COL_COMMENT, FORMAT_TYPE DATA_TYPE from {NETEZZA_SCHEMA_STORE}'''

snowflake = f'''select "snowflake" WAREHOUSE, run_timestamp RUN_TIMESTAMP, iteration_name ITERATION_NAME, concat_ws(".", TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME) TABLE_NAME, ORDINAL_POSITION COL_ORDER, COLUMN_NAME COL_NAME, COMMENT COL_COMMENT, GENERATED_DATA_TYPE DATA_TYPE from (select * ,
   case
     lower(DATA_TYPE)
     when 'number' then concat(DATA_TYPE,"(",NUMERIC_PRECISION,",",NUMERIC_SCALE,")")
     when 'text' then concat(DATA_TYPE,"(",data_length,")")
     else DATA_TYPE
   end as GENERATED_DATA_TYPE
from {SNOWFLAKE_SCHEMA_STORE})x'''

oracle = f'''select "oracle" WAREHOUSE, run_timestamp RUN_TIMESTAMP, iteration_name ITERATION_NAME, concat_ws(".",OWNER,TABLE_NAME) TABLE_NAME, COLUMN_ID COL_ORDER, COLUMN_NAME COL_NAME, COMMENTS COL_COMMENT, GENERATED_DATA_TYPE DATA_TYPE from (select * ,
   case
     lower(DATA_TYPE)
     when 'number' then concat(DATA_TYPE,"(",DATA_PRECISION,",",DATA_SCALE,")")
     when 'varchar2' then concat(DATA_TYPE,"(",DATA_LENGTH,")")
     when 'nvarchar2' then concat(DATA_TYPE,"(",DATA_LENGTH,")")
     else DATA_TYPE
   end as GENERATED_DATA_TYPE
from {ORACLE_SCHEMA_STORE})x'''

mssql = f'''select "mssql" WAREHOUSE, run_timestamp RUN_TIMESTAMP, iteration_name ITERATION_NAME, concat_ws(".",TABLE_SCHEMA,TABLE_NAME) TABLE_NAME, ORDINAL_POSITION COL_ORDER, COLUMN_NAME COL_NAME, COLUMN_COMMENT COL_COMMENT, DATA_TYPE DATA_TYPE from {MSSQL_SCHEMA_STORE}'''

teradata = f'''select "teradata" WAREHOUSE, run_timestamp RUN_TIMESTAMP, iteration_name ITERATION_NAME, concat_ws(".",DatabaseName,TableName) TABLE_NAME, ColumnOrder COL_ORDER, ColumnName COL_NAME, CommentString COL_COMMENT, GENERATED_DATA_TYPE DATA_TYPE  from (select * ,
  case
    lower(DATA_TYPE)
    when 'decimal' then concat(DATA_TYPE,"(",DecimalTotalDigits,",",DecimalFractionalDigits,")")
    when 'byte' then concat(DATA_TYPE,"(",ColumnLength,")")
    when 'varbyte' then concat(DATA_TYPE,"(",ColumnLength,")")
    when 'char' then concat(DATA_TYPE,"(",ColumnLength,")")
    when 'varchar' then concat(DATA_TYPE,"(",ColumnLength,")")
    else DATA_TYPE
  end as GENERATED_DATA_TYPE 
from {TERADATA_SCHEMA_STORE})x'''

hive = f'''select "hive" WAREHOUSE, run_timestamp RUN_TIMESTAMP, iteration_name ITERATION_NAME, concat_ws(".",db_name,table_name) TABLE_NAME, original_order COL_ORDER, col_name COL_NAME, comment COL_COMMENT, data_type DATA_TYPE from {HIVE_SCHEMA_STORE}'''
 

if spark.catalog.tableExists(DATABRICKS_SCHEMA_STORE):
  sqls.append(databricks)
if spark.catalog.tableExists(NETEZZA_SCHEMA_STORE):
  sqls.append(netezza)
if spark.catalog.tableExists(SNOWFLAKE_SCHEMA_STORE):
  sqls.append(snowflake)
if spark.catalog.tableExists(ORACLE_SCHEMA_STORE):
  sqls.append(oracle)
if spark.catalog.tableExists(MSSQL_SCHEMA_STORE):
  sqls.append(mssql)
if spark.catalog.tableExists(TERADATA_SCHEMA_STORE):
  sqls.append(teradata)
if spark.catalog.tableExists(HIVE_SCHEMA_STORE):
  sqls.append(hive)

sql = " UNION ".join(sqls)
spark.sql(sql)

# spark.sql(f'''CREATE OR REPLACE TEMPORARY VIEW schema_store_catalog_v as
# select "databricks" WAREHOUSE, run_timestamp RUN_TIMESTAMP, iteration_name ITERATION_NAME, concat_ws(".",db_name,table_name) TABLE_NAME, original_order COL_ORDER, col_name COL_NAME, comment COL_COMMENT, data_type DATA_TYPE from {DATABRICKS_SCHEMA_STORE}
# UNION
# select "netezza" WAREHOUSE, run_timestamp RUN_TIMESTAMP, iteration_name ITERATION_NAME, concat_ws(".",DATABASE,schema,Name) TABLE_NAME, ATTNUM COL_ORDER, ATTNAME COL_NAME, DESCRIPTION COL_COMMENT, FORMAT_TYPE DATA_TYPE from {NETEZZA_SCHEMA_STORE}
# UNION
# select "snowflake" WAREHOUSE, run_timestamp RUN_TIMESTAMP, iteration_name ITERATION_NAME, concat_ws(".", TABLE_CATALOG, TABLE_SCHEMA, TABLE_NAME) TABLE_NAME, ORDINAL_POSITION COL_ORDER, COLUMN_NAME COL_NAME, COMMENT COL_COMMENT, GENERATED_DATA_TYPE DATA_TYPE from (select * ,
#    case
#      lower(DATA_TYPE)
#      when 'number' then concat(DATA_TYPE,"(",NUMERIC_PRECISION,",",NUMERIC_SCALE,")")
#      when 'text' then concat(DATA_TYPE,"(",CHARACTER_MAXIMUM_LENGTH,")")
#      else DATA_TYPE
#    end as GENERATED_DATA_TYPE
# from {SNOWFLAKE_SCHEMA_STORE})x
# ''')

In [0]:
%python
dbutils.widgets.combobox("01-Iteration Name","", [iteration_name[0] for iteration_name in spark.sql(f"select iteration_name from (select distinct iteration_name, run_timestamp from {DATABRICKS_SCHEMA_STORE} order by run_timestamp desc)x").collect()])
iteration_name=dbutils.widgets.get("01-Iteration Name")
dbutils.widgets.dropdown("03-Additional Filters for Mismatch Chart only","N/A", [addtnl_filter[0] for addtnl_filter in spark.sql(f"SELECT distinct addtnl_filter from {MISMATCH_METRICS}").collect()])
addtnl_filter=dbutils.widgets.get("03-Additional Filters for Mismatch Chart only")
dbutils.widgets.combobox("02-Table Family","", [table_family[0] for table_family in spark.sql(f"SELECT distinct table_family from {MISMATCH_METRICS}").collect()])
table_family=dbutils.widgets.get("02-Table Family")

In [0]:
%python
display(spark.sql(f'''select workflow_name, table_family, src_table, tgt_warehouse, tgt_table, tgt_primary_keys from (with src_details as (
        select
        a.*,
        b.warehouse src_warehouse,
        b.jdbc_options src_jdbc_options
        from
        (select * from
        {VALIDATION_MAPPING_TABLE} where table_family in ('{table_family}'))a
        left outer join {TABLE_CONFIG_TABLE} b on a.src_connection_name = b.connection_name
        )
        select
        a.*,
        b.warehouse tgt_warehouse,
        b.jdbc_options tgt_jdbc_options
        from
        src_details a
        left outer join {TABLE_CONFIG_TABLE} b on a.tgt_connection_name = b.connection_name)x where table_family ='{table_family}' '''))

Databricks visualization. Run in Databricks to view.

In [0]:
%python
def readValidationTableList(validation_mapping_table,table_config_table):
  config_sql = f'''
        with src_details as (
        select
        a.*,
        b.warehouse src_warehouse,
        b.jdbc_options src_jdbc_options
        from
        (select * from
        {validation_mapping_table} where table_family in ('{table_family}'))a
        left outer join {table_config_table} b on a.src_connection_name = b.connection_name
        )
        select
        a.*,
        b.warehouse tgt_warehouse,
        b.jdbc_options tgt_jdbc_options
        from
        src_details a
        left outer join {table_config_table} b on a.tgt_connection_name = b.connection_name'''
  
  table_mappings = spark.sql(config_sql).collect()[0]
  return table_mappings

table_mapping = readValidationTableList(VALIDATION_MAPPING_TABLE,TABLE_CONFIG_TABLE)
src_wrhse = table_mapping["src_warehouse"]
tgt_wrhse = table_mapping["tgt_warehouse"]
src_tbl = table_mapping["src_table"]
tgt_tbl = table_mapping["tgt_table"]
table_family = table_mapping["table_family"]
print(src_wrhse, tgt_wrhse, src_tbl, tgt_tbl)

In [0]:
%python
mismatch_exclude_fields = table_mapping["mismatch_exclude_fields"]
if 'replicate_io_split_key' in mismatch_exclude_fields.split("|"):
  print("Blaa")
else:
  print("Bl")

In [0]:
%python
display(spark.sql(f'''select * from (select *, total_record_count-distinct_key_count as primary_key_violations from {PRIMARY_KEY_VALIDATION}  where iteration_name ="{iteration_name}" and lower(table_family) = lower("{table_family}") )a where primary_key_violations !=0'''))

Databricks visualization. Run in Databricks to view.

#Schema Validation
* Compares the Column Names, Column Comments, Data Types of the Source and Target Tables
* Also checks the compatibility between the datatypes that have mismatches 

In [0]:
%python
spark.sql(f'''
    select col_name, 
    IF(array_contains (split('{mismatch_exclude_fields}','\\\|'),tgt_col_name) OR array_contains (split('{mismatch_exclude_fields}','\\\|'),src_col_name), concat("BYPASS_",col_name_compare),col_name_compare) as col_name_compare, 
    IF(array_contains (split('{mismatch_exclude_fields}','\\\|'),tgt_col_name) OR array_contains (split('{mismatch_exclude_fields}','\\\|'),src_col_name), concat("BYPASS_",data_type_compare),data_type_compare) as data_type_compare, 
    IF(array_contains (split('{mismatch_exclude_fields}','\\\|'),tgt_col_name) OR array_contains (split('{mismatch_exclude_fields}','\\\|'),src_col_name), concat("BYPASS_",comment_compare),comment_compare) as comment_compare, 
    IF(array_contains (split('{mismatch_exclude_fields}','\\\|'),tgt_col_name) OR array_contains (split('{mismatch_exclude_fields}','\\\|'),src_col_name), concat("BYPASS_",datatype_congruency),datatype_congruency) as datatype_congruency
    from (                      
    select *,
            IF(
        lower(x.src_COL_NAME) <=> lower(x.tgt_COL_NAME),
        'MATCH',
        'MISMATCH'
    ) as col_name_compare,
    IF(
        lower(x.src_DATA_TYPE) <=> lower(x.tgt_DATA_TYPE),
        'MATCH',
        'MISMATCH'
    ) as data_type_compare,
    IF(
        lower(x.src_COL_COMMENT) <=> lower(x.tgt_COL_COMMENT),
        'MATCH',
        'MISMATCH'
    ) as comment_compare,
    if(
        rlike(
            lower(src_data_type),
            {src_wrhse}_compatibility_regex),
        true,
        false
        )as data_type_compatibility,
    if(y.has_precision,  
    if(
        (
        regexp_extract(
            lower(tgt_data_type),
            {src_wrhse}_compatibility_regex,
            2
        )
        ) -(
        regexp_extract(
            lower(src_data_type),
            {src_wrhse}_compatibility_regex,
            2
        )
        ) >= 0,
        true,
        false
    ),true) precision_compatible,
    if(y.has_precision,
        if(
        (
        nvl(int(regexp_extract(
            lower(tgt_data_type),
            {src_wrhse}_compatibility_regex,
            4
        )),0)
        ) -(
        nvl(int(regexp_extract(
            lower(src_data_type),
            {src_wrhse}_compatibility_regex,
            4
        )),0)
        ) >= 0,
        true,
        false
    ),true) scale_compatible,
    (data_type_compatibility and precision_compatible and scale_compatible) final_compatibility,
    IF(
        final_compatibility,
        'MATCH',
        'MISMATCH'
    ) as datatype_congruency
    from (select a.iteration_name, a.table_name src_table_name, b.table_name tgt_table_name, coalesce(a.col_name,b.col_name) as col_name, a.col_name src_col_name, b.col_name tgt_col_name, coalesce(a.col_order, b.col_order) col_order, a.data_type src_data_type, b.data_type tgt_data_type, a.col_comment src_col_comment, b.col_comment tgt_col_comment from (select * from schema_store_catalog_v where lower(warehouse) = lower('{src_wrhse}') and iteration_name ="{iteration_name}" and lower(TABLE_NAME) = lower("{src_tbl}"))a
    FULL OUTER JOIN 
    (select * from schema_store_catalog_v where lower(warehouse) = lower('{tgt_wrhse}') and iteration_name ="{iteration_name}" and lower(TABLE_NAME) =lower("{tgt_tbl}"))b
    ON lower(a.COL_NAME) = lower(b.COL_NAME)
    -- where lower(a.DATA_TYPE) <> lower(b.DATA_TYPE)
    order by COL_ORDER) x
    left outer join (select data_type lkp_data_type, warehouse, data_type data_type_aliases, has_precision, {src_wrhse}_compatibility_regex from {DATA_TYPE_COMPATIBILITY_MATRIX}
    where lower(warehouse) = lower('{tgt_wrhse}')) y
    on rlike(lower(x.tgt_data_type),lkp_data_type)
    -- where {src_wrhse}_compatibility_regex is not null
    )q order by COL_ORDER
    ''').display()

In [0]:
%python
spark.sql(f'''select a.iteration_name, a.table_name src_table_name, b.table_name tgt_table_name, coalesce(a.col_name,b.col_name) as col_name, a.col_name src_col_name, b.col_name tgt_col_name, a.data_type src_data_type, b.data_type tgt_data_type, a.col_comment src_col_comment, b.col_comment tgt_col_comment from (select * from schema_store_catalog_v where lower(warehouse) = lower('{src_wrhse}') and iteration_name ="{iteration_name}" and lower(TABLE_NAME) = lower("{src_tbl}"))a
    FULL OUTER JOIN 
    (select * from schema_store_catalog_v where lower(warehouse) = lower('{tgt_wrhse}') and iteration_name ="{iteration_name}" and lower(TABLE_NAME) =lower("{tgt_tbl}"))b
    ON lower(a.COL_NAME) = lower(b.COL_NAME)
    -- where lower(a.DATA_TYPE) <> lower(b.DATA_TYPE)
    order by b.COL_ORDER''').display()

In [0]:
%python
display(spark.sql(f'''
SELECT
  a.ITERATION_NAME,
  a.TABLE_NAME SRC_TABLE_NAME,
  b.TABLE_NAME TGT_TABLE_NAME,
  a.COL_NAME SRC_COL_NAME,
  b.COL_NAME TGT_COL_NAME,
  IF(
    lower(a.COL_NAME) <=> lower(b.COL_NAME),
    'MATCH',
    'MISMATCH'
  ) as col_name_compare
from
(select * from schema_store_catalog_v where lower(warehouse) = lower('{src_wrhse}') and iteration_name ="{iteration_name}" and lower(TABLE_NAME) = lower("{src_tbl}"))a
FULL OUTER JOIN 
(select * from schema_store_catalog_v where lower(warehouse) = lower('{tgt_wrhse}') and iteration_name ="{iteration_name}" and lower(TABLE_NAME) = lower("{tgt_tbl}"))b
ON a.COL_ORDER = b.COL_ORDER
order by a.COL_ORDER
'''))

Databricks visualization. Run in Databricks to view.

In [0]:
%python
display(spark.sql(f'''
SELECT
  a.ITERATION_NAME,
  a.TABLE_NAME SRC_TABLE_NAME,
  b.TABLE_NAME TGT_TABLE_NAME,
  a.COL_NAME SRC_COL_NAME,
  b.COL_NAME TGT_COL_NAME,
  IF(
    lower(a.COL_NAME) <=> lower(b.COL_NAME),
    'MATCH',
    'MISMATCH'
  ) as col_name_compare,
  a.DATA_TYPE SRC_DATA_TYPE,
  b.DATA_TYPE TGT_DATA_TYPE,
  IF(
    lower(a.DATA_TYPE) <=> lower(b.DATA_TYPE),
    'MATCH',
    'MISMATCH'
  ) as data_type_compare,
  a.COL_COMMENT SRC_COL_COMMENT,
  b.COL_COMMENT TGT_COL_COMMENT,
  IF(
    lower(a.COL_COMMENT) <=> lower(b.COL_COMMENT),
    'MATCH',
    'MISMATCH'
  ) as comment_compare
from
(select * from schema_store_catalog_v where lower(warehouse) = lower('{src_wrhse}') and iteration_name ="{iteration_name}" and lower(TABLE_NAME) = lower("{src_tbl}"))a
FULL OUTER JOIN 
(select * from schema_store_catalog_v where lower(warehouse) = lower('{tgt_wrhse}') and iteration_name ="{iteration_name}" and lower(TABLE_NAME) = lower("{tgt_tbl}"))b
ON lower(a.COL_NAME) = lower(b.COL_NAME)
order by a.COL_ORDER
'''))

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
%python
display(spark.sql(f'''
select *, if(
      rlike(
        lower(src_data_type),
        src_wrhse_compatibility_regex),
      true,
      false
    )as data_type_compatibility,
  if(y.has_precision,  
  if(
    (
      regexp_extract(
        lower(tgt_data_type),
        tgt_wrhse_compatibility_regex,
        2
      )
    ) -(
      regexp_extract(
        lower(src_data_type),
        src_wrhse_compatibility_regex,
        2
      )
    ) >= 0,
    true,
    false
  ),true) precision_compatible,
  if(y.has_precision,
    if(
    (
      nvl(int(regexp_extract(
        lower(tgt_data_type),
        tgt_wrhse_compatibility_regex,
        4
      )),0)
    ) -(
      nvl(int(regexp_extract(
        lower(src_data_type),
        src_wrhse_compatibility_regex,
        4
      )),0)
    ) >= 0,
    true,
    false
  ),true) scale_compatible,
  (data_type_compatibility and precision_compatible and scale_compatible) final_compatibility
  from (select a.iteration_name, a.table_name src_table_name, b.table_name tgt_table_name, b.col_name, a.data_type src_data_type, b.data_type tgt_data_type from (select * from schema_store_catalog_v where lower(warehouse) = lower('{src_wrhse}') and iteration_name ="{iteration_name}" and lower(TABLE_NAME) = lower("{src_tbl}"))a
FULL OUTER JOIN 
(select * from schema_store_catalog_v where lower(warehouse) = lower('{tgt_wrhse}') and iteration_name ="{iteration_name}" and lower(TABLE_NAME) =lower("{tgt_tbl}"))b
ON lower(a.COL_NAME) = lower(b.COL_NAME)
-- where lower(a.DATA_TYPE) <> lower(b.DATA_TYPE)
order by b.COL_ORDER) x
left outer join (select data_type lkp_data_type, warehouse, data_type data_type_aliases, has_precision, {src_wrhse}_compatibility_regex src_wrhse_compatibility_regex, {tgt_wrhse}_compatibility_regex tgt_wrhse_compatibility_regex from {DATA_TYPE_COMPATIBILITY_MATRIX}
 where lower(warehouse) = lower('{tgt_wrhse}')) y
on rlike(lower(x.tgt_data_type),lkp_data_type)
where src_wrhse_compatibility_regex is not null
'''))





Databricks visualization. Run in Databricks to view.

In [0]:
%python
display(spark.sql(f'''select SRC_COL_NAME as stage1, TGT_COL_NAME as stage2, SRC_DATA_TYPE as stage3, TGT_DATA_TYPE as stage4, count(TGT_DATA_TYPE) as value from (
SELECT
  a.ITERATION_NAME,
  a.TABLE_NAME SRC_TABLE_NAME,
  b.TABLE_NAME TGT_TABLE_NAME,
  a.COL_NAME SRC_COL_NAME,
  b.COL_NAME TGT_COL_NAME,
  IF(
    lower(a.COL_NAME) <=> lower(b.COL_NAME),
    'MATCH',
    'MISMATCH'
  ) as col_name_compare,
  a.DATA_TYPE SRC_DATA_TYPE,
  b.DATA_TYPE TGT_DATA_TYPE,
  IF(
    lower(a.DATA_TYPE) <=> lower(b.DATA_TYPE),
    'MATCH',
    'MISMATCH'
  ) as data_type_compare,
  a.COL_COMMENT SRC_COL_COMMENT,
  b.COL_COMMENT TGT_COL_COMMENT,
  IF(
    lower(a.COL_COMMENT) <=> lower(b.COL_COMMENT),
    'MATCH',
    'MISMATCH'
  ) as comment_compare
from
(select * from schema_store_catalog_v where lower(warehouse) = lower('{src_wrhse}') and iteration_name ="{iteration_name}" and lower(TABLE_NAME) = lower("{src_tbl}"))a
FULL OUTER JOIN 
(select * from schema_store_catalog_v where lower(warehouse) = lower('{tgt_wrhse}') and iteration_name ="{iteration_name}" and lower(TABLE_NAME) =lower("{tgt_tbl}"))b
ON lower(a.COL_NAME) = lower(b.COL_NAME)
order by a.COL_ORDER) group by all order by all
'''))

Databricks visualization. Run in Databricks to view.

In [0]:
%python
display(spark.sql(f"select * from {MISMATCH_METRICS} where iteration_name = '{iteration_name}' and  table_family ='{table_family}' order by mismatches desc"))

Databricks visualization. Run in Databricks to view.

In [0]:
%python
display(spark.sql(f"select * from {MISMATCH_METRICS} where iteration_name = '{iteration_name}' and table_family ='{table_family}' and addtnl_filter ='{addtnl_filter}' order by mismatches desc"))

Databricks visualization. Run in Databricks to view.

In [0]:
%python
dbutils.widgets.remove("04-Columns with Mismatch")

In [0]:
%python
dbutils.widgets.dropdown("04-Columns with Mismatch","*", [mismatch[0] for mismatch in spark.sql(f"select mismatch from (select distinct col_name as mismatch, mismatches from {MISMATCH_METRICS} where mismatches >0 and iteration_name ='{iteration_name}' and table_family ='{table_family}' order by mismatches desc)x").collect()]+["*"])

columns_with_mismatch=dbutils.widgets.get("04-Columns with Mismatch")



In [0]:
%python

columns_with_mismatch_where_condition = "" if columns_with_mismatch =="*" else f"and col_name ='{columns_with_mismatch}'"
mismatch_data_tbl=spark.sql(f"select concat(validation_data_db,'.',table_family,'__mismatch_data') as mismatch_data from {VALIDATION_MAPPING_TABLE} where table_family ='{table_family}'").collect()[0]["mismatch_data"]
if(spark.catalog.tableExists(mismatch_data_tbl)):
  display(spark.sql(f"select * from {mismatch_data_tbl} where iteration_name ='{iteration_name}' {columns_with_mismatch_where_condition}"))


Databricks visualization. Run in Databricks to view.

In [0]:
%python
display(spark.sql(f"select table_family, tgt_primary_keys, mismatch_exclude_fields from {VALIDATION_MAPPING_TABLE} where table_family='{table_family}'"));

Databricks visualization. Run in Databricks to view.

In [0]:
%python
display(spark.sql(f'''select * from {WINDOWED_VALIDATION_METRICS} where iteration_name ="{iteration_name}" and lower(table_family) = lower("{table_family}") order by last_updated_date, table_type'''))

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
%python
display(spark.sql(f'''
select
  a.table_family,
  tgt_tbl,
  a.iteration_name,
  a.addtnl_filter,
  count(col_name) total_columns, 
  sum(is_mismatch) mismatched_columns
from
  (
    select
      table_family,
      tgt_tbl,
      iteration_name,
      col_name,
      total_matches,
      mismatches,
      addtnl_filter,
      if(mismatches = 0, 0, 1) is_mismatch
    from
      {MISMATCH_METRICS}
      where table_family ='{table_family}' and iteration_name ='{iteration_name}'
  ) a
group by
  all
order by mismatched_columns desc
'''))

Databricks visualization. Run in Databricks to view.

In [0]:
%python
anomalies_data_tbl=spark.sql(f"select concat(validation_data_db,'.',table_family,'__anomalies') as anomalies_data from {VALIDATION_MAPPING_TABLE} where table_family ='{table_family}'").collect()[0]["anomalies_data"]
if(spark.catalog.tableExists(anomalies_data_tbl)):
  display(spark.sql(f"select '{table_family}' as table_family, * from {anomalies_data_tbl} where iteration_name ='{iteration_name}'"))


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.